# Programa de Pós-graduação em Recursos Hídricos e Sanemento - PPGRHS
## Hidrologia

**Evaporação**

Clebson Farias

## **Questão 1.** Em 20 de maio de 2015, a estação meteorológica localizada no Aeroporto Zumbi dos Palmares em Rio Largo registrou as seguintes leituras: temperatura do ar de $17^{o}C$, umidade relativa de $83\%$ e velocidade do vento de $0,9 m/s$. Sabendo disso, determine a taxa de evaporação potencial pelo Método Aerodinâmico naquele dia.

### Evaporação(Método Aerodinâmico)

In [2]:
temp_ar = 17
umidade_r = 0.83
veloc_vento = 0.9

#### Método Aerodinâmico
\begin{equation}
    E = B ⋅ (e_{s} − e_{a})
\end{equation}

Onde:

- $E$ -> E = Evaporação (mm/dia);
- $B$ -> Parâmetro em que é introduzido o efeito do vento por expressões empíricas;
- $e_{s}$ -> Pressão de vapor saturado;
- $e_{a}$ -> Pressão de vapor do ar.

#### Parâmetro em que é introduzido o efeito do vento por expressões empíricas
\begin{equation}
    B = \frac{0.102 · u}{[\ln(\frac{z_{2}}{z_{1}})]^2}
\end{equation}

Onde:

- $B$ -> Parâmetro em que é introduzido o efeito do vento por expressões empíricas;
- $u$ -> Velocidade do vento na altura $z_{2}$ (m/s);
- $z_{2}$ -> Altura da medição da velocidade do vento (geralmente é adotado 2 m a partir da superfície);
- $z_{1}$ -> Altura de rugosidade da superfície natural.

In [3]:
def tensao_vapor_saturado(temperatura, unidade_saida):
    """
    temperatura: Temperatura em °C
    unidade_saida: Unidade de Saída
    """
    A = {'mmHg': 4.58, 
         'Pa': 610.8}
    
    return round(A[unidade_saida] * 10 ** (7.5 * temperatura/(237.3 + temperatura)), 4)

pressao_sat = tensao_vapor_saturado(temp_ar, 'Pa')
print('Pressão de vapor saturado', pressao_sat, 'Pa')

Pressão de vapor saturado 1937.6501 Pa


In [5]:
def tensao_vapor_ar(umidade_relativa, tensao_vapor_sat):
    return round(umidade_relativa * tensao_vapor_sat, 4)

pressao_ar = tensao_vapor_ar(umidade_r, pressao_sat)
print('Pressão de vapor parcial: ', pressao_ar, 'Pa')

Pressão de vapor parcial:  1608.2496 Pa


In [10]:
import math

altura_medicao_vento = 2
altura_rugo_superficie = 0.02

def parametro_efeito(velocidade_vento, altura_medicao_vento, altura_rugo_superficie):
    b = (0.102 * velocidade_vento)/(math.log(altura_medicao_vento/altura_rugo_superficie) ** 2)
    return round(b, 4)

def evaporacao_aerod(pressao_sat, pressao_ar, velocidade_vento, altura_medicao_vento, altura_rugo_superficie):
    e = parametro_efeito(velocidade_vento, altura_medicao_vento, altura_rugo_superficie) * (pressao_sat - pressao_ar)
    return round(e, 4)

evaporacao = evaporacao_aerod(pressao_sat, pressao_ar, veloc_vento, altura_medicao_vento, altura_rugo_superficie)
print('Evaporação: ', evaporacao, 'mm/dia')

Evaporação:  1.4164 mm/dia


## **Questão 2.** No exemplo resolvido em sala de aula, calculamos a evaporação potencial pelo Balanço de Energia a partir de um valor de radiação média diária de $52 W/m^{2}$ e temepratura do ar de $17^{o}C$. Use a Equação de Penman para determinar a evaporação potencial considerando que ambos os fatores aerodinâmico e energético foram importantes para definir o total evaporado naquele dia.

In [34]:
radiacao_m = 52
temp_ar = 17

### Evaporação (Equação de Penman)

#### Equação de Penman
\begin{equation}
    E = [(\frac{\Delta}{\Delta + \gamma}) · E_{r} + (\frac{\gamma}{\Delta + \gamma}) · E_{a}]
\end{equation}

Onde:

- $E$ -> Evaporação (mm/dia);
- $E_{r}$ -> Evaporação calculada pelo método do balanço de energia (mm/dia);
- $E_{a}$ -> Evaporação calculada pelo método aerodinâmico (mm/dia);
- $\Delta$ -> Gradiente da curva de pressão de saturação de vapor (Pa/°C);
- $\gamma$ -> Constante psicrométrica (66,8 Pa/°C);

In [45]:
def evaporacao_pen(evap_energia, evap_aero, grad_pressao_sat, cons_psicrometrica):
    e1 = (grad_pressao_sat/(grad_pressao_sat + cons_psicrometrica)) * evap_energia
    e2 = (cons_psicrometrica/(grad_pressao_sat + cons_psicrometrica)) * evap_aero
    return round(e1 + e2, 4)

#### Gradiente da curva de pressão de saturação de vapor
\begin{equation}
    \Delta = \frac{4098 · e_{s}}{(237,3 + T) · 2}
\end{equation}

Onde:

- $\Delta$ -> Gradiente da curva de pressão de saturação de vapor (Pa/°C);
- $T$ -> Temperatura (°C);
- $e_{s}$ -> Pressao de vapor saturado (Pa);

In [46]:
def grad_pressao_vapor(pressao_vapor_sat, temperatura):
    delta = (4098 * pressao_vapor_sat)/((237.3 + temperatura) * 2)
    return round(delta, 4)

#### Evaporação calculada pelo método do balanço de energia
\begin{equation}
    E = \frac{\lambda_{u}}{\rho_{H_{2}O} · I_{u}}
\end{equation}

Onde:

- $E$ -> Evaporação (m/s)
- $\lambda_{u}$ -> Radiação liquída ($w/m^{2}$);
- $I_{u}$ -> Calor latente de vaporização (J/Kg);
- $\rho_{H_{2}O}$ -> Densidade de água ($kg/m^{3}$);

In [51]:
def evaporação_bal(radiacao_liq, calor_latente, densidade_agua):
    e = (86.4 * 10 ** 6) * (radiacao_liq/(calor_latente * densidade_agua))
    return round(e, 4)

#### Radiação liquída
\begin{equation}
    I_{u} = 2501 · 10^{3} - 2370 · T
\end{equation}

Onde:

- $I_{u}$ -> Calor latente de vaporização (J/Kg);
- $T$ -> Temperatura (°C)

In [52]:
def calor_latente(temperatura):
    r = 2501 * 10 ** 3 - 2370 * temperatura
    return round(r, 4)

In [54]:
#Evaporação de método aerodinâmico
e_aero = evaporacao_aerod(pressao_sat, pressao_ar, veloc_vento, altura_medicao_vento, altura_rugo_superficie)
e_energia = evaporação_bal(radiacao_m, calor_latente(temp_ar), 999)
e_pen = evaporacao_pen(e_energia, e_aero, grad_pressao_vapor(pressao_sat, temp_ar), cons_psicrometrica=66.8)
print('Evaporação: ', e_pen, 'mm/dia')

Evaporação:  1.8258 mm/dia
